Sobre los grupos resultantes podemos aplicar directamente cualquiera de las funciones de agregación y  estadísticas que ya te hemos presentado.

Pero hay mucho más. Disponemos de varios métodos para realizar distintos tipos de operaciones sobre cada grupo.

#### Agregados sobre grupos

Empecemos por el método `aggregate()`. Este método nos permite calcular múltiples valores agregados de forma simultánea, indicando en una lista las funciones de agregación a utilizar (ya sean proporcionadas por la librería o definidas por el usuario).

In [ ]:
# Agrupamos por ciudad
# y calculamos los valores de media y mediana
# de temperatura y velocidad de viento para cada ciudad
meteo_mes.groupby('ciudad')['temp_c','viento_vel_kmh'].aggregate(['mean', np.median, lambda x: min(x)])

temp_c                 viento_vel_kmh                
                mean median <lambda>           mean median <lambda>
ciudad                                                             
Barcelona  17.012500  16.00      9.0      14.250000  14.45      5.0
Bilbao     15.404167  15.90      8.0       8.862500   8.70      6.1
La Coruña  14.412500  14.45      9.3      10.437500  10.20      5.8
Madrid     15.637500  14.85      4.4      10.079167  10.35      5.4
Malaga     19.083333  18.20     11.4      11.495833  11.20      8.5
Sevilla    19.183333  18.75      9.6       9.266667   9.00      6.4
Valencia   18.275000  17.65     10.1      10.529167  10.00      5.0
Zaragoza   15.429167  15.15      6.0      16.670833  17.45      5.9

¿Has prestado atención cómo hemos usado el método `aggregate()`? Este ejemplo te muestra distintas formas de especificar las funciones de agregación a aplicar. En el primer elemento hemos escrito una cadena de texto para referirnos a la media. Pandas permite hacer esto con las operaciones comunes incluidas, él se encarga de traducir la cadena de texto a la función correspondiente. En el segundo elemento hemos utilizado directamente el nombre de una función y no una cadena de texto (en este caso el método para calcular la mediana de la librería NumPy). Y en el último elemento incluimos una expresión _lambda_ a medida (también podríamos haber puesto el nombre de una función definida por nosotros.

¿Qué me dices del resultado? Ha creado un índice jerárquico para las columnas. En el primer nivel tienes los nombres de las columnas originales y en el segundo nivel el nombre de los agregados que hemos calculado.

#### Filtrado de grupos

El método `filter()` nos permite decidir si queremos incluir los datos de un grupo o no, en base a una función de filtrado sobre el conjunto de valores de cada grupo. La función de filtrado debe devolver un valor booleano para cada valor de entrada.

Por ejemplo, vamos a filtrar y descartar las ciudades cuya velocidad de viento promedio supere los 12km/h

In [ ]:
meteo_mes.groupby('ciudad').filter(lambda x: x['viento_vel_kmh'].mean() < 12).head()

,año,mes,ciudad,temp_c,viento_vel_kmh
1,2015,1,Bilbao,9.1,8.7
2,2015,1,La Coruña,9.6,10.8
3,2015,1,Madrid,4.4,9.0
4,2015,1,Malaga,11.4,13.6
5,2015,1,Sevilla,9.6,8.9


Ten en cuenta que el resultado de esta operación es un DataFrame como el original, pero sin los datos de los grupos excluidos.

#### Transformación de datos por grupos

Cuando calculamos valores agregados por grupos, estamos reduciendo el DataFrame de cada grupo con sus múltiples filas u observaciones a una sola fila con los resultados de la agregación.

Pero podemos estar interesados en mantener los registros o filas originales, y simplemente modificar sus valores en base a las características de cada grupo. Un ejemplo típico sería la normalización de variables por grupos. Para este tipo de operaciones utilizamos el método `transform()`.

In [ ]:
# Normalizamos la variable `temperatura` por grupos
Z_temp_c = meteo_mes.groupby('ciudad')['temp_c'].transform(lambda x: (x - x.mean())/x.std())

pd.concat([meteo_mes, Z_temp_c], axis = 'columns').head()

,año,mes,ciudad,temp_c,viento_vel_kmh,temp_c
0,2015,1,Barcelona,9.1,17.7,-1.402767
1,2015,1,Bilbao,9.1,8.7,-1.436302
2,2015,1,La Coruña,9.6,10.8,-1.330867
3,2015,1,Madrid,4.4,9.0,-1.405386
4,2015,1,Malaga,11.4,13.6,-1.406828


La función que usemos debe devolver un valor de salida simple para cada valor de entrada. El resultado de `transform()` es un objeto Series con un valor por cada fila (temperatura) del DataFrame original.

#### Aplicando funciones arbitrarias

Si lo que queremos es devolver directamente un DataFrame por cada grupo, podemos utilizar el método `apply()`. El argumento debe ser una función que recibirá el DataFrame correspondiente a un grupo y que devolverá un nuevo DataFrame arbitrario como resultado.

In [ ]:
# Función que añade una nueva columna 
# con la variable temperatura normalizada
def fnorm(x):
    x['Z_temp_c'] = (x['temp_c'] - x['temp_c'].mean())/x['temp_c'].std()
    return x

# Aplicar la función por grupos
meteo_mes.groupby('ciudad').apply(fnorm).head()


,año,mes,ciudad,temp_c,viento_vel_kmh,Z_temp_c
0,2015,1,Barcelona,9.1,17.7,-1.402767
1,2015,1,Bilbao,9.1,8.7,-1.436302
2,2015,1,La Coruña,9.6,10.8,-1.330867
3,2015,1,Madrid,4.4,9.0,-1.405386
4,2015,1,Malaga,11.4,13.6,-1.406828
